# 📖 Chapter 02 — Data Processing & Chunking

## 🎯 Objectives

In this chapter, we will process the raw tourism data into a format suitable for RAG implementation.

We will create unified document structures, implement chunking strategies, and save processed data for vector database ingestion.

In [7]:
import pandas as pd
from src.config import RAW_DATA_DIR, PROCESSED_DATA_DIR
import json
from langchain_text_splitters import RecursiveCharacterTextSplitter
from src.utils.emoji_log import step, task, data, info, success, save, done

## 🧹 Step 01 — Data Loading

Load the raw tourism data from JSON file.

In [8]:
with open(RAW_DATA_DIR / "scenic_spot.json", "r", encoding="utf-8-sig") as f:
    loaded_data = json.load(f)

attraction = loaded_data["XML_Head"]["Infos"]["Info"]
df = pd.DataFrame(attraction)

text_cols = ["Add", "Description", "Opentime", "Ticketinfo", "Travellinginfo"]
df[text_cols] = df[text_cols].fillna("")

## 📄 Step 02 — Document Creation

Transform each attraction into a unified document format.

In [9]:
def create_document(row):
    # Combine the text columns
    content_parts = [
        f"Name: {row['Name']}",
        f"Region: {row['Region']}",
    ]

    # if column exists, return it, otherwise, return "", and "" is false in if
    if str(row.get("Add", "")).strip():
        content_parts.append(f"Address: {row['Add']}")

    if str(row.get("Description", "")).strip():
        content_parts.append(f"Description: {row['Description']}")

    if str(row.get("Opentime", "")).strip():
        content_parts.append(f"Open_time: {row['Opentime']}")

    if str(row.get("Ticketinfo", "")).strip():
        content_parts.append(f"Ticket: {row['Ticketinfo']}")

    # Filter the empty
    content = "\n".join([part for part in content_parts if part])

    return {
        "id": row["Id"],
        "content": content,
        "metadata": {
            "name": row["Name"],
            "region": row["Region"],
            "category": row.get("Class1", "Unknown"),
        },
    }

In [10]:
contents = df.apply(
    create_document, axis=1
).tolist()  # apply manipulates the columns default.

## ✂️ Step 03 — Text Chunking

Test chunking strategies for longer documents.

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=[
        "\n\n",
        "\n",
        " ",
        "",
    ],  # Prefer to separate at paragraph, new line, and space
)

In [12]:
sample_doc = contents[0]
info("Original document:")
print(sample_doc["content"])
data(f"Length: {len(sample_doc['content'])} characters")

💬 Original document:
Name: 宏亞食品巧克力觀光工廠
Region: 桃園市
Address: 桃園縣八德市建國路386號
Description: 巧克力共和國是一座以巧克力為主題的觀光工廠，建築設計、館內主題設計皆以巧克力為主題，這裡也提供豐富的巧克力相關知識，亦可以DIY創作巧克力，為一寓教於樂、適合親子休閒娛樂的絕佳去處。
Ticket: 收費方式請電洽
📊 Length: 173 characters


In [13]:
# Chunk
chunks = text_splitter.split_text(sample_doc["content"])
print(f"Separate to {len(chunks)} chunks")

for i, chunk in enumerate(chunks, 1):
    print(f"\n--- Chunk {i} ({len(chunk)} characters) ---")
    print(chunk)

Separate to 1 chunks

--- Chunk 1 (173 characters) ---
Name: 宏亞食品巧克力觀光工廠
Region: 桃園市
Address: 桃園縣八德市建國路386號
Description: 巧克力共和國是一座以巧克力為主題的觀光工廠，建築設計、館內主題設計皆以巧克力為主題，這裡也提供豐富的巧克力相關知識，亦可以DIY創作巧克力，為一寓教於樂、適合親子休閒娛樂的絕佳去處。
Ticket: 收費方式請電洽


In [14]:
doc_lengths = [len(doc["content"]) for doc in contents]
length_df = pd.DataFrame({
    "length": doc_lengths
})

print("Docs length summary:")
print(length_df.describe())

Docs length summary:
            length
count  5086.000000
mean    204.130751
std     135.558116
min      21.000000
25%      85.000000
50%     175.000000
75%     287.750000
max     643.000000


In [15]:
need_chunking = sum(1 for length in doc_lengths if length > 500)
data(f"Documents over 500 characters: {need_chunking} / {len(contents)}")
info(f"Ratio: {need_chunking / len(contents) * 100:.1f}%")

📊 Documents over 500 characters: 99 / 5086
💬 Ratio: 1.9%


## 💾 Step 04 — Save Processed Data

Export processed documents to JSON format.

In [16]:
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

output_path = PROCESSED_DATA_DIR / "documents.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(contents, f, ensure_ascii=False, indent=2)

save(f"Saved {len(contents)} documents to: {output_path}")
data(f"File size: {output_path.stat().st_size / 1024 / 1024:.2f} MB")

💾 Saved 5086 documents to: c:\Users\dinni\OneDrive\桌面\Travel_rag\data\processed\documents.json
📊 File size: 3.21 MB


In [17]:
# Double check
with open(output_path, "r", encoding="utf-8") as f:
    loaded_docs = json.load(f)

success("Verification complete!")
data(f"Original documents: {len(contents)}")
data(f"Loaded documents: {len(loaded_docs)}")
success(f"Data consistency: {len(contents) == len(loaded_docs)}")
done("Chapter 02 completed!")

✅ Verification complete!
📊 Original documents: 5086
📊 Loaded documents: 5086
✅ Data consistency: True
🏁 Chapter 02 completed!
